In [129]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif

from sklearn.neural_network import MLPRegressor


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline 

import helper_RF as l

In [130]:
### Multi-layer Perceptron
mod = MLPRegressor(random_state=0)

scal = RobustScaler()
select_k_best = SelectKBest(f_classif)
m_pipe = make_pipeline(scal, select_k_best, mod)

m_param_grid = {'selectkbest__k':range(5, 25),
                'mlpregressor__activation':['relu', 'logistic'], 
                'mlpregressor__learning_rate':['adaptive'],
                'mlpregressor__solver':['lbfgs', 'adam'],
                'mlpregressor__hidden_layer_sizes':[(33,33,33)]}
#mod.early_stopping=True # terminate training when validation score is not improving
mod.shuffle=True # shuffle samples when training


mlp = GridSearchCV(m_pipe, m_param_grid, cv=3, n_jobs=-1)